In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
load_dotenv()

True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish?"

In [3]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
memory_agent = agents.memory_agent()

# create tasks
context_task = tasks.determine_memory_task(context_agent, user_context)
memory_task = tasks.store_memory(memory_agent)

memory_task.context = [context_task]
    
crew = Crew(
    agents=[
        context_agent, 
        memory_agent
    ],
    tasks=[
        context_task,
        memory_task
    ]
)

In [4]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: Allergy: Fish

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to utilize the insert_memory tool to store the memory provided by the context_specialist in the database.

Action: 
insert_memory
Action Input: 
{"property_type": "Allergy", "property_value": "Fish"} 

None

Final Answer: The memory "Allergy: Fish" has been successfully stored in the database.

> Finished chain.


In [5]:
from memo_store import MemoStore
memo_store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
memo_store.list_memos()


LOADING MEMORY FROM DISK
    Location = ./tmp/teachable_agent_db/uid_text_dict.pkl
LIST OF MEMOS
  ID: 1
    INPUT TEXT: Allergy
    OUTPUT TEXT: Fish


In [ ]:
# memo_store.reset_db()